# Elexon API

First iteration.
See API documentation [here](https://www.elexon.co.uk/wp-content/uploads/2017/06/bmrs_api_data_push_user_guide_v1.1.pdf).

### Dependencies

* `pandas`

### How To Use

* store your API key in `api_key.txt`
* set the start and end date
* set the signal key (see attached csv file)
* enjoy

In [1]:
import requests
from io import StringIO
import pandas as pd

In [11]:
# store your api_key in a separate file
with open('api_key.txt', 'r') as f:
    api_key = f.read()

api_url = 'https://api.bmreports.com/'

output_format='csv'

In [4]:
def generate_api_request(api_key, signal_key, settlement_date, period=None, output_format='csv', api_url="https://api.bmreports.com/", version='v1'):
    '''
    Creates the url request for the Elexon API
    https://api.bmreports.com/BMRS/B1760/%3CVersionNo%3E?APIKey=%3CAPIKey%3E&SettlementDate=%3CSettlementDate
    
    Args
    -----        
    *-----------------*--------*------------*----------------------------*
    | Argument        | type   | Format     | Example                    |
    *-----------------*--------*------------*----------------------------*
    | api_url         | String | NA         | https://api.bmreports.com/ |
    | singnal_key     | String | NA         | B1760                      |
    | api_key         | String | NA         | AP8DA23                    |
    | settlement_date | String | YYYY-MM-DD | 2014-12-31                 |
    | period          | String | */1-50     | 1                          |
    | output_format   | String | NA         | csv/xml                    |
    *-----------------*--------*------------*----------------------------*
        
    '''
    
    # determine period description ("*" for all periods in the day)
    if period is not None:
        if isinstance(period, int):
            period_str = str(period)
        else:
            raise Exception('period must be None (for whole day) or an integer')
    else:
        period_str = '*'
    
    r_url = '''{}BMRS/{}/{}?APIKey={}&SettlementDate={}&Period={}&ServiceType={}'''.format(api_url, signal_key, version, api_key, 
                                                                                          settlement_date.strftime('%Y-%m-%d'), 
                                                                                          period_str, output_format)
            
    return r_url

In [8]:
signal_key = 'B1760'

start=pd.datetime(2018, 1,15)
end  =pd.datetime(2018, 1,17)

In [9]:
df_l = []

for settlement_date in pd.DatetimeIndex(start=start, end=end, freq='d'):
    
    r_url = generate_api_request(api_key, signal_key, settlement_date, output_format=output_format)
    r = requests.get(r_url)
    
    if not r.ok: raise Exception('''API call failed with status code {}. \n\n Full message: {}'''.format(r.status_code, r.text))
        
    data = StringIO(r.text.split('\n*\n*')[-1].replace('<EOF>',''))
    _df = pd.read_csv(data)
    
    df_l.append(_df)

df = pd.concat(df_l, axis=0, ignore_index=True)

In [10]:
df.head()

,Time Series ID,Business Type,Control Area,Power System Resource Type,Flow Direction,Settlement Date,Settlement Period,Activation Price Amount(GBP),Price Category,Doc Status,Curve Type,Resolution,Document Type,Process Type,Active Flag,Document ID,Document RevNum
0,NGET-EMFIP-PABE-TS-00450123,Replacement reserve,10YGB----------A,Generation,UP,2018-01-15,48,0.0,NaN,Final,Sequential fixed size block,PT30M,Activated balancing prices,Realised,Y,NGET-EMFIP-PABE-00057237,1
1,NGET-EMFIP-PABE-TS-00450122,Replacement reserve,10YGB----------A,Load,DOWN,2018-01-15,48,0.0,NaN,Final,Sequential fixed size block,PT30M,Activated balancing prices,Realised,Y,NGET-EMFIP-PABE-00057237,1
2,NGET-EMFIP-PABE-TS-00450121,Manual frequency restoration reserve,10YGB----------A,Generation,UP,2018-01-15,48,0.0,NaN,Final,Sequential fixed size block,PT30M,Activated balancing prices,Realised,Y,NGET-EMFIP-PABE-00057237,1
3,NGET-EMFIP-PABE-TS-00450120,Manual frequency restoration reserve,10YGB----------A,Load,DOWN,2018-01-15,48,0.0,NaN,Final,Sequential fixed size block,PT30M,Activated balancing prices,Realised,Y,NGET-EMFIP-PABE-00057237,1
4,NGET-EMFIP-PABE-TS-00450119,Automatic frequency restoration reserve,10YGB----------A,Load,DOWN,2018-01-15,48,0.0,NaN,Final,Sequential fixed size block,PT30M,Activated balancing prices,Realised,Y,NGET-EMFIP-PABE-00057237,1
